In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from statsmodels.tsa.statespace.sarimax import SARIMAX 
from statsmodels.tsa.arima.model import ARIMA
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller 
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf 
from sklearn.metrics import mean_absolute_error, mean_squared_error 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from datetime import datetime
from scipy import stats
from collections import defaultdict


In [ ]:
# install pmdarima for model estimation
! pip install pmdarima

In [ ]:
from pmdarima.arima import auto_arima

In [ ]:

ru_equip_deaths_df = pd.read_csv("../data/russia_losses_equipment.csv")
ru_equip_deaths_df.head()

ru_pers_deaths_df = pd.read_csv("../data/russia_losses_personnel.csv")
ru_pers_deaths_df.head()

# May not get to this. incorporate a global list that holds all T-Test results across all tests
global_Ttest = defaultdict(list)

## Data is cumulative.
###

### This data is read from most recent to least recent. 
### for ease of understanding we will reverse this

In [ ]:
ru_equip_deaths_df = ru_equip_deaths_df.sort_index(ascending=False)
ru_equip_deaths_df.reset_index(inplace=True)
ru_equip_deaths_df

ru_pers_deaths_df = ru_pers_deaths_df.sort_index(ascending=False)
ru_pers_deaths_df.reset_index(inplace=True)
ru_pers_deaths_df

In [ ]:
#EDA scatter matrix.  uncomment the below lines to take a look at potential relationships
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
# scatter_matrix(ru_equip_deaths_df,figsize=(12,12), diagonal="kde");
# plt.tight_layout()

In [ ]:
ru_equip_deaths_df["tank"].diff(1).plot.line(y="day")
# monthly_mean_tank_loss = 


In [ ]:

ru_pers_deaths_df["personnel"].diff(1).plot.line(y="day")
# rework and relabel these plots

In [ ]:
# stationarity check for our time series data. This will be important for our SARIMA model 

def check_stationarity(timeseries): 
    # Perform the Dickey-Fuller test 
    result = adfuller(timeseries, autolag='AIC') 
    p_value = result[1] 
    print(f'ADF Statistic: {result[0]}') 
    print(f'p-value: {p_value}') 
    print('Stationary' if p_value < 0.05 else 'Non-Stationary') 

check_stationarity(ru_pers_deaths_df["personnel"].diff(1).dropna())


In [ ]:
ru_pers_deaths_df["personnel"].diff(1).dropna()


In [ ]:
ru_equip_deaths_df["vehicles and fuel tanks"].iloc[0]

In [ ]:
ru_equip_deaths_df.info()
# We know only the first 64 entries have nans. merge the data from these two fields
for i in range(65):
    if pd.isna(ru_equip_deaths_df["vehicles and fuel tanks"].iloc[i]):
        ru_equip_deaths_df["vehicles and fuel tanks"].iloc[i] = ru_equip_deaths_df["fuel tank"].iloc[i] \
        + ru_equip_deaths_df["military auto"].iloc[i] 

ru_equip_deaths_df.info()


## Create Diff columns

In [ ]:
def create_diff_columns(df_name, column_name):
    """
    Create a new dataframe column for the daily differences in loss rates 
    args: pandas dataframe, str name for column


    """
    df_name[f"{column_name} diff"] = df_name[f"{column_name}"].diff(1)
    df_name[f"{column_name} diff"][0] = df_name[f"{column_name}"][0]

# these are the four I will focus on 
create_diff_columns(ru_equip_deaths_df, "vehicles and fuel tanks")
create_diff_columns(ru_equip_deaths_df, "tank")
create_diff_columns(ru_equip_deaths_df, "APC")
create_diff_columns(ru_equip_deaths_df, "field artillery")
create_diff_columns(ru_pers_deaths_df, "personnel")



# Create columns for a Datetime object and for the int month and int year for calculations
#these are only made once for each csv
ru_equip_deaths_df["Dt_OBJ"] = ru_equip_deaths_df['date'].apply(lambda x : datetime.strptime(x, '%Y-%m-%d'))
ru_equip_deaths_df["Int_month"] = ru_equip_deaths_df["Dt_OBJ"].dt.month
ru_equip_deaths_df["Int_year"] = ru_equip_deaths_df["Dt_OBJ"].dt.year

ru_pers_deaths_df["Dt_OBJ"] = ru_pers_deaths_df['date'].apply(lambda x : datetime.strptime(x, '%Y-%m-%d'))
ru_pers_deaths_df["Int_month"] = ru_pers_deaths_df["Dt_OBJ"].dt.month
ru_pers_deaths_df["Int_year"] = ru_pers_deaths_df["Dt_OBJ"].dt.year


# For season calculations we fake the year december is in so visualizations won't be a problem
#actual datetime value will not change or interfere with data
for i in range(len(ru_equip_deaths_df)):
        if ru_equip_deaths_df["Int_month"].iloc[i] == 12:
            ru_equip_deaths_df.iloc[i,-1] += 1 

In [ ]:
ru_equip_deaths_df["field artillery diff"].plot.line(y="day")

## Groupby months to evaluate each monthly trend

#### We'll use this to guide our seasonality checks

In [ ]:
# currently deprecated other than creating the DT objects. 

# ru_equip_deaths_df["Dt_OBJ"] = ru_equip_deaths_df['date'].apply(lambda x : datetime.strptime(x, '%Y-%m-%d'))

# # ru_equip_deaths_df.groupby(pd.Grouper(key='date', freq='M'))

# # type(ru_equip_deaths_df["Dt_OBJ"])
# # # ru_equip_deaths_df["Dt_OBJ"]
# # # ru_equip_deaths_df.head()
# # def monthly_analysis(df_obj, c_name):
# #     by_month_df = ru_equip_deaths_df.groupby(pd.Grouper(key='Dt_OBJ', freq='M'))["tank diff"]
    




# # ru_by_month_df = ru_equip_deaths_df.groupby(pd.Grouper(key='Dt_OBJ', freq='M'))
# # ru_by_month_df.sum()["tank diff"].plot.line()
# # ru_by_month_df.sum()["tank diff"]


# ru_pers_deaths_df["pers diff"] = ru_pers_deaths_df["personnel"].diff(1)
# ru_pers_deaths_df["pers diff"][0] = 2800
# # ru_pers_deaths_df["pers diff"]

# ru_pers_deaths_df["Dt_OBJ"] = ru_pers_deaths_df['date'].apply(lambda x : datetime.strptime(x, '%Y-%m-%d'))
# ru_pers_deaths_df.head()

# ru_pers_by_month_df = ru_pers_deaths_df.groupby(pd.Grouper(key='Dt_OBJ', freq='M'))
# ru_pers_by_month_df.sum()["pers diff"]
# ru_pers_deaths_df['Dt_OBJ'][1].month


# Create season dataframes

In [ ]:
def return_seasonal_dataframes(df_name, column_name):
    '''Only return season dataframes
    Args: dataframe, str for column
    returns four dataframes in order spring, summer, fall, winter
    '''
    springdf = df_name[(df_name["Int_month"] == 3) | (df_name["Int_month"] == 4) |
                                                (df_name["Int_month"] == 5)]
    summerdf = df_name[(df_name["Int_month"] == 6) | (df_name["Int_month"] == 7) |
                                                (df_name["Int_month"] == 8)]
    falldf = df_name[(df_name["Int_month"] == 9) | (df_name["Int_month"] == 10) |
                                                (df_name["Int_month"] == 11)]
    winterdf = df_name[(df_name["Int_month"] == 12) | (df_name["Int_month"] == 1) |
                                                (df_name["Int_month"] == 2)]
    
    return springdf,summerdf,falldf,winterdf

def create_seasonal_comparisons(df_name, column_name):
    # split the given dataframe into seasons, plot the dataframes
    # run T-Tests against spring and print results

    
    #df_name["Int_month"] = df_name["Dt_OBJ"].dt.month

    ru_deaths_summerdf = df_name[(df_name["Int_month"] == 6) | (df_name["Int_month"] == 7) |
                                                (df_name["Int_month"] == 8)]
    ru_deaths_falldf = df_name[(df_name["Int_month"] == 9) | (df_name["Int_month"] == 10) |
                                                (df_name["Int_month"] == 11)]
    ru_deaths_winterdf = df_name[(df_name["Int_month"] == 12) | (df_name["Int_month"] == 1) |
                                                (df_name["Int_month"] == 2)]
    ru_deaths_springdf = df_name[(df_name["Int_month"] == 3) | (df_name["Int_month"] == 4) |
                                                (df_name["Int_month"] == 5)]
    
    ru_deaths_summerdf[f"{column_name}"].plot()
    ru_deaths_springdf[f"{column_name}"].plot()
    ru_deaths_falldf[f"{column_name}"].plot()
    ru_deaths_winterdf[f"{column_name}"].plot()
    print("Summer mean:", ru_deaths_summerdf[f"{column_name}"].mean())
    print("Spring mean:" ,ru_deaths_springdf[f"{column_name}"].mean())
    print("Fall mean:" ,ru_deaths_falldf[f"{column_name}"].mean())
    print("Winter mean:" ,ru_deaths_winterdf[f"{column_name}"].mean())
    
    print("Winter vs Spring:" , stats.ttest_ind(ru_deaths_winterdf[f"{column_name}"], ru_deaths_springdf[f"{column_name}"]))
    print("Summer vs Spring:" ,stats.ttest_ind(ru_deaths_summerdf[f"{column_name}"], ru_deaths_springdf[f"{column_name}"]))
    print("Fall vs Spring:" ,stats.ttest_ind(ru_deaths_falldf[f"{column_name}"], ru_deaths_springdf[f"{column_name}"]))

# Add color vairable, change seasonal plot


def trend_line_w_outliers(df_name, column_name, draw_data=False,color=("blue","yellow","green","grey"), draw_trend=True ):
    ''' slightly misleading, but calls for draw trend line do all the work
    split DF into seasons, with options to draw the data to a plot, the trend line, or both
    color expects a 4-tuple of string color names for plotting
    Args: dataframe, str, boolean, 4-len tuple, boolean
    '''
    
    ru_deaths_summerdf = df_name[(df_name["Int_month"] == 6) | (df_name["Int_month"] == 7) |
                                                (df_name["Int_month"] == 8)]
    ru_deaths_falldf = df_name[(df_name["Int_month"] == 9) | (df_name["Int_month"] == 10) |
                                                (df_name["Int_month"] == 11)]
    ru_deaths_winterdf = df_name[(df_name["Int_month"] == 12) | (df_name["Int_month"] == 1) |
                                                (df_name["Int_month"] == 2)]
    ru_deaths_springdf = df_name[(df_name["Int_month"] == 3) | (df_name["Int_month"] == 4) |
                                                (df_name["Int_month"] == 5)]
    plt.figure(figsize=(12, 6))

    summer_std = ru_deaths_summerdf[f"{column_name}"].std()
    fall_std = ru_deaths_falldf[f"{column_name}"].std()
    winter_std = ru_deaths_winterdf[f"{column_name}"].std()
    spring_std = ru_deaths_springdf[f"{column_name}"].std()
    if draw_data:
        plt.plot(df_name["Dt_OBJ"],df_name[f"{column_name}"])
    
    print("Spring:")
    draw_trend_line(ru_deaths_springdf,column_name,color[0],draw_trend)
    print("Summer:")
    draw_trend_line(ru_deaths_summerdf,column_name,color[1],draw_trend)
    print("Fall:")
    draw_trend_line(ru_deaths_falldf,column_name,color[2],draw_trend)
    print("Winter:")
    draw_trend_line(ru_deaths_winterdf,column_name,color[3],draw_trend)
   
    
    

def draw_trend_line(df_name, column_name, color_name, draw_trend=True):
    #Attach december to the next year for plotting purposes
    outlier_counter = 0


    yearly_data = df_name.groupby(df_name["Int_year"])
    df_name_mean = df_name[f"{column_name}"].median()
    
    for year, data in yearly_data:
        # print("yearly", year, data.head())
        df_name_std = data[f"{column_name}"].std()
        df_name_mean = data[f"{column_name}"].mean()
        # print(f"{year}:STD : {df_name_std}")
        # print(f"{year}:mean : {df_name_mean}")
        
        first_point = data.iloc[0]
        last_point = data.iloc[-1]
        if draw_trend:
            plt.plot([first_point["Dt_OBJ"], last_point["Dt_OBJ"]], 
                    [first_point[f'{column_name}'], last_point[f'{column_name}']], 
                    marker='o', color=color_name)
        
        for points in range(len(data)):
            
            if (data[f"{column_name}"].iloc[points] > (df_name_std * 2) + df_name_mean):
                #or (df_name[f"{column_name}"].iloc[points] < df_name_mean - df_name_std)
                # print(" Outlier at ",data["Int_month"].iloc[points],  data[f"{column_name}"].iloc[points])
                outlier_counter += 1
                plt.plot(data["Dt_OBJ"].iloc[points],data[f"{column_name}"].iloc[points], marker='x',color="black")

    
    plt.xlabel('Date')
    plt.ylabel('Value')
    plt.title('Trend Lines for Each Month')
    plt.legend()
    plt.grid(True)
    print(f"Total season outliers: {outlier_counter}")

# y = df['passengers']
# x = df['lag_1']
# model = sm.OLS(y, sm.add_constant(x))
# results = model.fit()
# b, m = results.params
# IQR, standard deviation to find outliers. plot them, take linear regression lines for each season

In [ ]:
# for i in range(len(ru_pers_deaths_df)):
#         if ru_pers_deaths_df["Int_month"].iloc[i] == 12:
#             ru_pers_deaths_df.iloc[i, ru_pers_deaths_df.columns.get_loc('Int_year')] += 1

trend_line_w_outliers(ru_pers_deaths_df, "personnel diff",draw_data=False)


# SNS.REGPLOT

In [ ]:
trend_line_w_outliers(ru_equip_deaths_df, "tank diff",draw_data=False,draw_trend=True)


In [ ]:
trend_line_w_outliers(ru_equip_deaths_df, "APC diff",draw_data=True,draw_trend=False)

In [ ]:
trend_line_w_outliers(ru_equip_deaths_df, "field artillery diff",draw_data=True,draw_trend=False)

In [ ]:
plt.figure(figsize=(12, 6))
ru_pers_deaths_df.plot.line(x="Dt_OBJ", y="personnel diff")

In [ ]:

create_seasonal_comparisons(ru_pers_deaths_df, "personnel diff")


In [ ]:
create_seasonal_comparisons(ru_equip_deaths_df, "APC diff")

In [ ]:
create_seasonal_comparisons(ru_equip_deaths_df, "field artillery diff")

In [ ]:

create_seasonal_comparisons(ru_equip_deaths_df, "vehicles and fuel tanks diff")

In [ ]:
def seasonal_decomposition(df_name, column_name):

    equip_analysis = df_name.loc[0::,["Dt_OBJ", f"{column_name} diff"]]
    equip_analysis.set_index("Dt_OBJ", inplace=True)
    decompose_result = seasonal_decompose(equip_analysis,period=90)
    trend = decompose_result.trend
    seasonal = decompose_result.seasonal
    resid_season = decompose_result.resid
    decompose_result.plot()
    return trend, seasonal, resid_season, decompose_result

apc_trend, apc_seasonal, apc_resid, APC_decomp = seasonal_decomposition(ru_equip_deaths_df,"APC")

In [ ]:
apc_trend.plot()
plt.axvline(x=datetime(2022,8,29),color="red")
plt.axvline(x=datetime(2023,9,1))
plt.axvline(x=datetime(2022,7,3),color="black")

In [ ]:
pers_trend, pers_seasonal, pers_resid, pers_decomp = seasonal_decomposition(ru_pers_deaths_df,"personnel")

In [ ]:
pers_trend.plot()
plt.axvline(x=datetime(2022,8,29),color="red")
plt.axvline(x=datetime(2023,9,1))
plt.axvline(x=datetime(2022,7,3),color="black")

In [ ]:
#testing lags at different areas

# group into seasons, compare means of seasons, drop back into days, t-test
# seasonal decomposition sarima

# outlier analysis

### Checks for stationarity

In [ ]:
check_stationarity(ru_equip_deaths_df["vehicles and fuel tanks diff"])

In [ ]:
ru_equip_deaths_df["vehicles and fuel tanks"].diff(1).dropna().plot.line(y="day")

In [ ]:
check_stationarity(ru_equip_deaths_df["tank"].diff(1).dropna())



In [ ]:
from pandas.plotting import autocorrelation_plot
autocorrelation_plot(ru_equip_deaths_df["tank"].diff(1).dropna())
autocorrelation_plot(ru_equip_deaths_df["tank"].dropna())


In [ ]:
autocorrelation_plot(ru_equip_deaths_df["vehicles and fuel tanks"].diff(1).dropna())

In [ ]:
ru_equip_deaths_df["tank"].diff(1).dropna().hist()
plt.close()

In [ ]:
ru_pers_deaths_df["personnel"].diff(-1).dropna().hist()
plt.close()

In [ ]:
autocorrelation_plot(ru_pers_deaths_df["personnel"].diff(-1).dropna())
plt.close()

In [ ]:
plot_acf(ru_equip_deaths_df["tank"].diff(1).dropna()) 
plot_pacf(ru_equip_deaths_df["tank"].diff(1).dropna()) 
plt.show()
plt.close()

In [ ]:
autocorrelation_plot(ru_equip_deaths_df["tank"].diff(1).dropna())
plt.close()

In [ ]:
# use pd.cut to bin by seasons (i.e. spring, summer, fall, winter), according to the ukrainian climate.
# hypothesis, there will be a statistically significant change in casualty rates 
# across vehicle and personnel in spring
# Furthermore that change will be lower than summer and fall casualty rates

In [ ]:
apc_spring_df, apc_summer_df, apc_fall_df, apc_winter_df = return_seasonal_dataframes(ru_equip_deaths_df,'APC diff')
plot_acf(ru_equip_deaths_df["APC diff"]) 
plot_pacf(ru_equip_deaths_df["APC diff"]) 
plt.show()
plt.close()

In [ ]:
plt.close()

# The SARIMA MODELS

## Code below here is Fitting and reviewing models.
##### These may be computationally expensive!

In [ ]:
# wrap in

from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error

#Split the train and test
X = ru_equip_deaths_df["APC diff"]
tscsv = TimeSeriesSplit()
for i, (train_index, test_index) in enumerate(tscsv.split(X)):
    X_train = ru_equip_deaths_df.iloc[train_index]
    y_train = ru_equip_deaths_df.iloc[train_index]["APC diff"]
    X_test = ru_equip_deaths_df.iloc[test_index]
    y_test = ru_equip_deaths_df.iloc[test_index]["APC diff"]

    # forecast

In [ ]:
# GRID SEARCH
# this would take a long time to implement, will run auto arima for a ballpark and try to manually find best fit
# make sure you have a large chunk of memory. searching for this m value is very expensive

Arima_model= auto_arima(y_train, start_p=1, 
                        start_q=1, 
                        max_p=8, 
                        max_q=8, 
                        start_P=0, 
                        start_Q=0, 
                        max_P=8, 
                        max_Q=8,
                        m=30, 
                        seasonal=True, 
                        trace=True, 
                        d=1, D=1, 
                        error_action='warn', 
                        suppress_warnings=True, 
                        random_state = 20, 
                        n_fits=30)


# def parameter_grid_search()

# def sarima_forecast(history, config):
#  order, sorder, trend = config
#  model = SARIMAX(history, order=order, seasonal_order=sorder, trend=trend, enforce_stationarity=False, enforce_invertibility=False)
 
#  model_fit = model.fit(disp=False)
#  # make one step forecast
#  yhat = model_fit.predict(len(history), len(history))
#  return yhat[0]

# def model_validation_stepper(df_name, n_test, cfg):
#     predictions = list()
#ARIMA(2,1,0)(7,1,0)[12] 


In [ ]:
Arima_model.fit(y_train)

forecaster = Arima_model.predict(len(y_test)-1)


In [ ]:
type(forecaster)

In [ ]:
forecaster.plot.line()

In [ ]:
#  ARIMA(2,1,0)(7,1,0)[12] 


# Define SARIMA parameters by results of autoarima.  worst case try these
p, d, q = 2, 1, 0 
P, D, Q, s = 7, 1, 0, 30   
  
# Fit the SARIMA model 
# ar_model = ARIMA(ru_equip_deaths_df["tank"].diff(1).dropna(), order=(p,d,q))
model = SARIMAX(y_train, order=(p, d, q), seasonal_order=(P, D, Q, s)) 
results = model.fit() 


In [ ]:
predictions = results.predict(start=0, end=len(y_train)-1)



In [ ]:
forecast = results.get_forecast(steps=90)

forecast_df = pd.DataFrame(forecast.predicted_mean)
forecast_df

In [ ]:
results.summary()

In [ ]:
# fig, ax = plt.subplots()
forecast= results.get_forecast(steps=len(y_test)-1)
forecast_df = pd.DataFrame(forecast.predicted_mean)
forecast_df

In [ ]:
inp = input()
if inp in ru_equip_deaths_df.columns:
    print("it work")

In [ ]:


# plt.plot(y_train.index,y_test[1])
plt.plot()
y_train.plot.line()
plt.plot(forecaster.index,forecaster.values)
y_test.plot.line()



#### losses per day cannot be negative
##### Roll through predictions and set negative predicts to zero

In [ ]:
predictions[predictions < 0] = 0


In [ ]:
ru_equip_deaths_df["APC diff"].plot.line()
predictions.plot.line()


In [ ]:
residuals = ru_equip_deaths_df["APC diff"] - predictions
residuals.dropna(inplace=True)

residuals = pd.DataFrame(residuals)
residuals.reset_index()
residuals.columns = ['diff']


fig, ax = plt.subplots(figsize=(14, 4))
# ax.scatter(range(len(predictions)), predictions, color="red", label="y true values")
ax.scatter(range(len(residuals)), residuals, color="k", label='y_hat')
# for i in range(1, len(predictions)):
#     plt.vlines(i,residuals['diff'][i],predictions[i],color="k", linestyle="--")

    # 0,y_hat[0],y[0],color="k", linestyle="--"
ax.legend()
plt.title("AVG EXP")
plt.show()


# AIC, BIC

In [ ]:
cas_residuals = results.resid
cas_residuals


In [ ]:

predictions[predictions < 0] = 0
plt.scatter(predictions[1::], cas_residuals[1::])
plt.axhline(y=0,color="orange",linestyle="--")
plt.show()


In [ ]:
autocorrelation_plot(cas_residuals)

In [ ]:
ru_equip_deaths_df["tank"].diff(1).plot.line(y="day")

In [ ]:
ru_equip_deaths_df["tank"].plot.line(y="day")
ru_equip_deaths_df["tank"]
plt.close()